In [50]:
import pandas as pd
url="https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data"
AB=pd.read_csv(url, header=None, names=['Sex','Length','Diameter','Height','Whole','Shucked','Viscera','Shell','Rings'])
print(AB.head())
X=AB[['Length','Diameter','Height','Whole','Shucked','Viscera','Shell','Rings']]
y=AB[['Sex']]

import numpy as np
y_ab=np.where(y=="I",1,0)
print(np.unique(y_ab, return_counts=True)) # (array([0, 1]), array([2835, 1342]))

  Sex  Length  Diameter  Height   Whole  Shucked  Viscera  Shell  Rings
0   M   0.455     0.365   0.095  0.5140   0.2245   0.1010  0.150     15
1   M   0.350     0.265   0.090  0.2255   0.0995   0.0485  0.070      7
2   F   0.530     0.420   0.135  0.6770   0.2565   0.1415  0.210      9
3   M   0.440     0.365   0.125  0.5160   0.2155   0.1140  0.155     10
4   I   0.330     0.255   0.080  0.2050   0.0895   0.0395  0.055      7
(array([0, 1]), array([2835, 1342]))


In [51]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y_ab, stratify=y, test_size=0.2, random_state=42)

In [52]:
print(x_train.shape, x_test.shape)
print(np.unique(y_train, return_counts=True))

(3341, 8) (836, 8)
(array([0, 1]), array([2268, 1073]))


In [47]:
x_train = np.array(x_train)
y_train = np.array(x_train)

In [57]:
class SingleLayer:
  def __init__(self):
    self.w = None
    self.b = None
    self.losses = []

  def forpass(self, x):
    z = np.sum(x * self.w) + self.b # 직선 방정식을 계산
    return z

  def backprop(self, x, err):
    w_grad = x * err # 가중치에 대한 그레이디언트를 계산
    b_grad = 1 * err # 절편에 대한 그레이디언트를 계산
    return w_grad, b_grad

  def add_bias(self, X):
    return np.c_[np.ones((X.shape[0], 1)), X] # 행렬의 맨 앞에 1로 채워진 열 벡터를 추가

  def activation(self, z):
    a = 1 / (1 + np.exp(-z)) # 시그모이드 계산
    return a

  def fit(self, x, y, epochs=100):
    if isinstance(x, (pd.DataFrame, pd.Series)):
        x = x.values
    if isinstance(y, (pd.DataFrame, pd.Series)):
        y = y.values
    self.w = np.ones(x.shape[1]) # 가중치를 초기화
    self.b = 0 # 절편을 초기화
    for i in range(epochs): # epochs만큼 반복
      loss = 0
# 인덱스를 섞습니다
      indexes = np.random.permutation(np.arange(len(x)))
      for i in indexes: # 모든 샘플에 대해 반복
        z = self.forpass(x.iloc[i]) # 정방향 계산
        a = self.activation(z) # 활성화 함수 적용
        err = -(y[i] - a) # 오차 계산
        w_grad, b_grad = self.backprop(x.iloc[i], err) # 역방향 계산
        self.w -= w_grad # 가중치 업데이트
        self.b -= b_grad # 절편 업데이트
# 안전한 로그 계산을 위하여 클리핑한 후 손실을 누적함
        a = np.clip(a, 1e-10, 1-1e-10)
        loss += -(y[i]*np.log(a)+(1-y[i])*np.log(1-a))
# 에포크마다 평균 손실을 저장
      self.losses.append(loss/len(y))

  def predict(self, x):
    z = [self.forpass(x_i) for x_i in x] # 정방향 계산
    return np.array(z) > 0 # 스텝 함수 적용

  def predict(self, x):
    z = [self.forpass(x_i) for x_i in x] # 정방향 계산
    return np.array(z) > 0 # 스텝 함수 적용

  def predict(self, x):
      z = [self.forpass(x_i) for x_i in x]
      return np.array(z) > 0

  def score(self, x, y):
      return np.mean(self.predict(x) == y)


layer = SingleLayer()
layer.fit(x_train, y_train)
print(layer.score(x_test, y_test))

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [61]:
X = X.astype(float)

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(X, y_ab, stratify=y_ab, test_size=0.3, random_state=42)

class SingleLayer:
    def __init__(self):
        self.w = None
        self.b = None
        self.losses = []

    def forpass(self, x):
        z = np.sum(x * self.w) + self.b
        return z

    def backprop(self, x, err):
        w_grad = x * err
        b_grad = 1 * err
        return w_grad, b_grad

    def activation(self, z):
        a = 1 / (1 + np.exp(-z))
        return a

    def fit(self, x, y, epochs=100):
        self.w = np.ones(x.shape[1])
        self.b = 0
        for _ in range(epochs):
            loss = 0
            indexes = np.random.permutation(np.arange(len(x)))
            for i in indexes:
                z = self.forpass(x.iloc[i])
                a = self.activation(z)
                err = -(y[i] - a)
                w_grad, b_grad = self.backprop(x.iloc[i], err)
                self.w -= w_grad
                self.b -= b_grad
                a = np.clip(a, 1e-10, 1 - 1e-10)
                loss += -(y[i] * np.log(a) + (1 - y[i]) * np.log(1 - a))
            self.losses.append(loss / len(y))

    def predict(self, x):
        z = [self.forpass(x_i) for x_i in x.values]
        return np.array(z) > 0

    def score(self, x, y):
        return np.mean(self.predict(x) == y)

layer = SingleLayer()
layer.fit(x_train, y_train)
print(layer.score(x_test, y_test))

# 훈련 데이터의 정확도 출력
print(layer.score(x_train, y_train))

<ipython-input-61-64cafcec931e>:22: RuntimeWarning: overflow encountered in exp
  a = 1 / (1 + np.exp(-z))


0.7974481658692185
0.7991789257612042


In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np

# 데이터 로드 및 전처리
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data"
AB = pd.read_csv(url, header=None, names=['Sex', 'Length', 'Diameter', 'Height', 'Whole', 'Shucked', 'Viscera', 'Shell', 'Rings'])

# 'Sex' 열은 y로 처리하고, 나머지는 x로 처리
x = AB[['Length', 'Diameter', 'Height', 'Whole', 'Shucked', 'Viscera', 'Shell', 'Rings']]
y = AB['Sex']

# y 값을 이진 분류로 변환 ('I' = 1, 'M'/'F' = 0)
y_ab = np.where(y == "I", 1, 0)

# 데이터 분할 (train과 test로 나눔)
x_train, x_test, y_train, y_test = train_test_split(x, y_ab, stratify=y_ab, test_size=0.3, random_state=42)

# 스케일링 (SGD는 데이터 스케일에 민감하므로 표준화가 필요)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# SGDClassifier 모델 생성
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(x_train, y_train)

# 훈련 데이터 정확도
train_accuracy = sgd_clf.score(x_train, y_train)
print(f"Train data accuracy: {train_accuracy}")

# 테스트 데이터 정확도
test_accuracy = sgd_clf.score(x_test, y_test)
print(f"Test data accuracy: {test_accuracy}")


Train data accuracy: 0.7013342456380431
Test data accuracy: 0.7025518341307815
